In [1]:
!pip3 install docplex
!pip3 install pandas

In [2]:
import docplex.mp.model as cpx
import pandas as pd

In [3]:
def parse_data(csv_file):
  ''' Returns a clean pandas dataframe containing the stock data in csv_file '''
  df = pd.read_csv(csv_file)
  df = df.dropna(axis=1).set_index('Date', inplace=False)
  df.index = pd.to_datetime(df.index)
  df.sort_index(inplace=True)
  idx = pd.date_range(df.index[0], df.index[-1])
  df = df.reindex(idx, method='bfill')
  return df

In [4]:
df = parse_data('snp.csv')

In [5]:
index = parse_data('index.csv')

In [6]:
index

,Close/Last,Volume,Open,High,Low
2017-11-29,2626.07,--,2627.82,2634.89,2620.32
2017-11-30,2647.58,--,2633.93,2657.74,2633.93
2017-12-01,2642.22,--,2645.10,2650.62,2605.52
2017-12-02,2639.44,--,2657.19,2665.19,2639.03
2017-12-03,2639.44,--,2657.19,2665.19,2639.03
...,...,...,...,...,...
2022-11-24,4026.12,--,4023.34,4034.02,4020.76
2022-11-25,4026.12,--,4023.34,4034.02,4020.76
2022-11-26,3963.94,--,4005.36,4012.27,3955.77
2022-11-27,3963.94,--,4005.36,4012.27,3955.77


In [7]:
df.head()

,MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,ADP,...,WTW,GWW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
2017-01-03,178.050003,47.500000,39.049999,62.410000,112.360001,116.459999,36.639999,46.189999,103.480003,103.500000,...,123.239998,234.449997,87.459999,40.619999,49.650002,63.209999,86.250000,100.320389,43.180000,53.590000
2017-01-04,178.320007,47.919998,39.360001,63.290001,115.739998,116.739998,37.360001,46.110001,104.139999,103.660004,...,124.760002,236.330002,90.279999,40.799999,50.389999,63.439999,87.029999,101.242722,43.799999,54.110001
2017-01-05,177.710007,47.700001,39.700001,63.770000,114.809998,114.989998,37.939999,45.770000,105.910004,103.040001,...,125.959999,232.259995,91.440002,40.799999,49.930000,63.650002,84.750000,101.893204,43.090000,53.930000
2017-01-06,178.229996,47.720001,40.779999,63.790001,115.419998,116.300003,37.910000,44.720001,108.300003,103.110001,...,126.779999,231.500000,92.430000,40.919998,49.580002,64.419998,85.959999,101.902916,43.369999,54.099998
2017-01-07,177.270004,47.400002,40.740002,64.209999,117.110001,115.000000,37.700001,44.750000,108.570000,102.470001,...,126.010002,230.350006,92.750000,40.299999,49.369999,64.599998,85.970001,103.883492,42.900002,53.950001


In [8]:
returns = df.pct_change(7)[df.index.dayofweek == 0].dropna()
index_returns = index['Close/Last'].pct_change(7)[index.index.dayofweek == 0].dropna()

In [9]:
#returns = returns.iloc[:, :10]

In [10]:
index_returns

2017-12-11    0.007786
2017-12-18    0.011342
2017-12-25   -0.003591
2018-01-01    0.005712
2018-01-08    0.019252
                ...   
2022-10-31    0.019656
2022-11-07   -0.016834
2022-11-14    0.039521
2022-11-21   -0.001847
2022-11-28    0.003544
Name: Close/Last, Length: 260, dtype: float64

Variables:

$x_0, x_1,\dots, x_n$: weights of stocks in portfolio, sum should be 1

---

Returns defined as: $$r_{i,t} = \frac{P_{i,t} - P_{i,t-1}}{P_{i,t-1}}$$
$P_{i,t}$ is the closing price for stock $i$ at day $t$.

$T=150$: 150 week in-sample size

### Omega Ratio

In [11]:
def expected_return(asset, returns_df):
    return returns_df[asset].mean()

In [12]:
def expected_return_portfolio(portfolio_weights, returns_df):
    output = 0
    for i in range(len(portfolio_weights)):
      output += expected_return(returns_df.columns[i], returns_df) * portfolio_weights[i]
    return output

In [13]:
opt_model = cpx.Model(name="Omega Model")

In [14]:
L = float(index_returns.mean())
T = 150
n = returns.shape[1]
p = 1./150
x_vars  = {n: opt_model.continuous_var(lb=0, ub= 1,
                                 name="x_{0}".format(n)) 
for n in range(n)}
theta = opt_model.continuous_var(lb=0, name="theta")
d_vars = {t: opt_model.continuous_var(name="d_{0}".format(t)) for t in range(T)}
z = opt_model.continuous_var(name="z")

constraints = {}

constraints[3] = opt_model.add_constraint(ct = opt_model.sum(opt_model.sum(float(returns.iloc[t, i])*x_vars[i]*p for i in range(n)) for t in range(T)) - z*L >= theta, ctname="3")


for t in range(T):
    constraints["5_{0}".format(t)] = opt_model.add_constraint(ct = L*z - opt_model.sum(float(returns.iloc[t, i])*x_vars[i] for i in range(n)) <= d_vars[t], ctname="5_{0}".format(t))

constraints[6] = opt_model.add_constraint(ct = opt_model.sum(p*d_vars[t] for t in range(T)) == 1, ctname="6")
constraints[7] = opt_model.add_constraint(ct = opt_model.sum(x_vars[i] for i in range(n)) == 1, ctname="7")
opt_model.maximize(theta)


In [15]:
#constraints[8] = opt_model.add_constraint(ct = z <= 1/p, ctname="8")

### Robust Omega Ratio

In [16]:
def cutting_plane(returns):
  ''' Applies the cutting plane algorithm using the given returns as data '''
  pass

In [17]:
solution = opt_model.solve(log_output=True)

Version identifier: 22.1.1.0 | 2023-06-15 | d64d5bd77
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 1 columns.
Reduced LP has 153 rows, 639 columns, and 74261 nonzeros.
Presolve time = 0.02 sec. (14.52 ticks)

Iteration log . . .
Iteration:     1   Scaled dual infeas =             1.000000
Perturbation started.
Iteration:    51   Scaled dual infeas =             1.000000
Iteration:   101   Dual objective     =             3.564429
Removing perturbation.


In [18]:
print(solution)

solution for: Omega Model
objective: 0.0275228
status: OPTIMAL_SOLUTION(2)
x_166=1.000
theta=0.028
d_2=0.124
d_3=0.047
d_7=0.284
d_8=0.096
d_10=0.075
d_12=0.015
d_13=0.030
d_14=0.023
d_15=0.062
d_16=0.025
d_17=0.342
d_19=0.038
d_21=0.013
d_25=0.035
d_26=0.036
d_29=0.074
d_30=0.057
d_33=145.072
d_36=0.099
d_38=0.065
d_39=0.014
d_40=0.064
d_46=0.117
d_48=0.052
d_49=0.055
d_51=0.073
d_52=0.133
d_53=0.057
d_57=0.041
d_63=0.083
d_64=0.080
d_65=0.046
d_67=0.163
d_72=0.010
d_73=0.202
d_78=0.091
d_79=0.001
d_80=0.155
d_82=0.068
d_83=0.074
d_84=0.036
d_86=0.096
d_87=0.120
d_90=0.067
d_92=0.049
d_93=0.117
d_96=0.026
d_99=0.017
d_100=0.026
d_101=0.161
d_109=0.022
d_114=0.066
d_117=0.063
d_118=0.032
d_121=0.004
d_124=0.006
d_127=0.015
d_130=0.016
d_132=0.003
d_137=0.200
d_138=0.141
d_139=0.023
d_141=0.184
d_145=0.017
d_146=0.294
d_148=0.007

